In [1]:
import sys
assert sys.version_info >= (3, 5)

import numpy as np
import pandas as pd
import os

import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import time

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [5]:
#x_train = train_data.iloc[:, 1:]
#x_train = x_train.values.astype(np.float32).reshape(-1, 28*28) / 255
#x_valid, x_train = x_train[:4200], x_train[4200:]

(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_valid = x_train_full[:-5000], x_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

x_mean = x_train.mean(axis = 0, keepdims = True)
x_std = x_train.std(axis = 0, keepdims = True) + 1e-7

x_train = (x_train - x_mean)/x_std
x_train = x_train[..., np.newaxis]
x_valid = (x_valid - x_mean)/x_std
x_valid = x_valid[..., np.newaxis]

#y_train = train_data.iloc[:, 0]
#y_train = y_train.values.astype(np.float32)
#y_valid, y_train = y_train[:4200], y_train[4200:]

#x_test = test_data.values.astype(np.float32).reshape(-1, 28*28) / 255
x_test = (x_test - x_mean)/x_std
x_test = x_test[..., np.newaxis]

In [6]:
from functools import partial

In [7]:
def_conv2d = partial(keras.layers.Conv2D, kernel_size = 3, activation = 'relu', padding = "SAME")

In [8]:
cnn = keras.models.Sequential([
    def_conv2d(filters = 64, kernel_size = 7, input_shape = [28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size = 2),
    def_conv2d(filters = 128),
    def_conv2d(filters = 128),
    keras.layers.MaxPooling2D(pool_size = 2),
    def_conv2d(filters = 256),
    def_conv2d(filters = 256),
    keras.layers.MaxPooling2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 64, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 10, activation = 'softmax')
])

In [9]:
cnn.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
history = cnn.fit(x_train, y_train, epochs = 10, validation_data = [x_valid, y_valid])

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 418s 8ms/sample - loss: 0.6862 - accuracy: 0.7626 - val_loss: 0.3830 - val_accuracy: 0.8642
Epoch 2/10
18688/55000 [=========>....................] - ETA: 4:23 - loss: 0.4282 - accuracy: 0.8568

KeyboardInterrupt: 